In [1]:
# Import required libraries
import cv2
import numpy as np
from IPython.display import HTML
from base64 import b64encode

# Function to compute median background
def median_background(frames):
    stacked_frames = np.stack(frames, axis=0)  # (num_frames, height, width)
    median_frame = np.median(stacked_frames, axis=0).astype(np.uint8)
    return median_frame

# Function to detect vehicles using median differencing
def detect_vehicles(video_path):
    print("Detecting Vehicles....")
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    max_frames = 50  # Number of frames used to estimate background

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray)
        frame_count += 1

    background = median_background(frames)

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    results = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(gray, background)
        _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
        contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            if cv2.contourArea(cnt) > 500:
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        results.append(frame)

    cap.release()
    return results

# Function to save the video
def save_video(frames, filename="output.avi", fps=20):
    print("Saving Video....")
    height, width, _ = frames[0].shape
    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()

# # Function to display video in Colab
# def display_video(filename):
#     mp4 = open(filename, 'rb').read()
#     data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#     return HTML(f"""<video width=640 controls><source src="{data_url}" type="video/mp4"></video>""")

# Main execution
processed_frames = detect_vehicles("Assets/traffic.mp4")
save_video(processed_frames, "traffic_output.avi")
#display_video("traffic_output.avi")


Detecting Vehicles....
Saving Video....
